In [ ]:
from pyspark.sql import SparkSession

# Spark session & context
spark = SparkSession.builder.appName("world-energy-stats").master("spark://spark-master:7077").getOrCreate()
sc = spark.sparkContext

# Sum of the first 100 whole numbers
rdd = sc.parallelize(range(100 + 1))
rdd.sum()
# 5050

In [ ]:
df = (spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
   .load("hdfs://namenode:9000/energy-data/owid-energy-data.csv"))

In [ ]:
df.head()

In [ ]:
df = df[df['year'] >=1990]

#40 years of data
grouped_df = df.groupBy("year").count().orderBy("year")
grouped_df.show(40)


In [ ]:
# Dropping irrelevant columns
cols_to_drop = [col for col in df.columns if '_per_capita' in col or '_change_pct' in col or '_change_twh' in col]
df = df.drop(*cols_to_drop)

# Show the updated DataFrame
df.show(n=2)